In [1]:
import fastreg as fr
from fastreg import I, R, C, D, B, factor

In [2]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [3]:
# generate random testing data
data = fr.dataset(N=100_000, K1=10, K2=100, models=['linear', 'poisson'])
data

,id1,id2,x1,x2,yhat0,yhat,yhat1,y0,y,Ep0,Ep,Ep1,p0,p,p1
0,E,37,-0.259664,0.957506,0.596604,1.366604,0.996604,0.140456,3.450027,1.815941,3.922009,2.709066,1,0,4
1,I,65,0.334285,-1.045791,-0.427189,1.022811,0.372811,-0.551597,0.954923,0.652340,2.781001,1.451810,2,1,1
2,I,10,0.067322,-0.631068,-0.258444,0.641556,0.541556,-0.682874,1.517074,0.772252,1.899434,1.718679,0,4,1
3,I,10,0.384297,0.485393,0.506525,1.406525,1.306525,-0.236097,3.559011,1.659514,4.081746,3.693317,3,2,5
4,G,38,-1.584996,-1.037097,-0.997757,-0.017757,-0.397757,-1.692442,-2.045029,0.368706,0.982400,0.671825,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,D,73,0.624556,0.857360,0.801783,1.831783,1.101783,-0.951971,2.520384,2.229513,6.245012,3.009527,2,5,6
99996,D,53,2.040068,-0.346070,0.504379,1.334379,0.804379,0.245231,0.334883,1.655956,3.797635,2.235307,2,5,2
99997,J,40,0.423945,0.758248,0.682132,1.982132,1.582132,-1.915105,1.672198,1.978091,7.258203,4.865319,3,7,6
99998,D,90,-0.232874,-0.372146,-0.193150,1.006850,0.106850,-0.774781,0.737252,0.824358,2.736966,1.112767,0,2,2


### Formulas

`Factor` — a column or expression

`Term` — a combination of `Factor`s

`Formula` — a combination of unique `Term`s

Built in `Factor`s are: `[R]eal`, `[C]ategorical`, `[I]ntercept`, `[D]emeaned`, and `[B]inned`

In [4]:
# pure [R]eal (a factor)
R('x1')

R(x1)

In [5]:
# convenience method for simple columns
R.x1

R(x1)

In [6]:
# interacted [R]eals (a term)
R.x1*R.x2

R(x1)*R(x2)

In [7]:
# add in [I]ntercept and [C]ategorical (a formula)
I + R.x1*R.x2 + C.id1

I + R(x1)*R(x2) + C(id1)

In [8]:
# pure [B]inned column
B('x1', 10)

C(x1:bin10)

In [9]:
# add in [B]inned and [D]emeaned (a formula)
I + B.x1(10)*R.x1 + D.x2('id1')

I + C(x1:bin10)*R(x1) + R(x2-μ|id1)

In [10]:
# use general python expressions in factors
I + R('abs(x1)') + C('x2 > 0')

I + R(abs(x1)) + C(x2 > 0)

### Interfaces

In [11]:
# pythonic formula
fr.ols(y=R.y, x=I+R.x1+R.x2+C.id1+C.id2, data=data)

y,coeff,stderr,low95,high95,pvalue
I,0.152789,0.033121,0.087873,0.217705,3.968445e-06
x1,0.295420,0.003172,0.289203,0.301637,0.000000e+00
x2,0.593774,0.003153,0.587595,0.599954,0.000000e+00
id1=B,0.071617,0.014167,0.043850,0.099384,4.298797e-07
id1=C,0.167662,0.014115,0.139997,0.195327,0.000000e+00
...,...,...,...,...,...
id2=95,0.863448,0.044494,0.776242,0.950654,0.000000e+00
id2=96,0.950444,0.045121,0.862008,1.038879,0.000000e+00
id2=97,0.961423,0.044398,0.874405,1.048441,0.000000e+00
id2=98,0.978801,0.044968,0.890666,1.066936,0.000000e+00


In [12]:
# R style formula
form = 'y ~ 1 + x1 + x2 + C(id1) + C(id2)'
fr.ols(formula=form, data=data)

y,coeff,stderr,low95,high95,pvalue
I,0.152789,0.033121,0.087873,0.217705,3.968445e-06
x1,0.295420,0.003172,0.289203,0.301637,0.000000e+00
x2,0.593774,0.003153,0.587595,0.599954,0.000000e+00
id1=B,0.071617,0.014167,0.043850,0.099384,4.298797e-07
id1=C,0.167662,0.014115,0.139997,0.195327,0.000000e+00
...,...,...,...,...,...
id2=95,0.863448,0.044494,0.776242,0.950654,0.000000e+00
id2=96,0.950444,0.045121,0.862008,1.038879,0.000000e+00
id2=97,0.961423,0.044398,0.874405,1.048441,0.000000e+00
id2=98,0.978801,0.044968,0.890666,1.066936,0.000000e+00


In [13]:
# tuple/list style formula
fr.ols(y='y', x=['1', 'x1', 'x2', C('id1'), C('id2')], data=data)

y,coeff,stderr,low95,high95,pvalue
1,0.152789,0.033121,0.087873,0.217705,3.968445e-06
x1,0.295420,0.003172,0.289203,0.301637,0.000000e+00
x2,0.593774,0.003153,0.587595,0.599954,0.000000e+00
id1=B,0.071617,0.014167,0.043850,0.099384,4.298797e-07
id1=C,0.167662,0.014115,0.139997,0.195327,0.000000e+00
...,...,...,...,...,...
id2=95,0.863448,0.044494,0.776242,0.950654,0.000000e+00
id2=96,0.950444,0.045121,0.862008,1.038879,0.000000e+00
id2=97,0.961423,0.044398,0.874405,1.048441,0.000000e+00
id2=98,0.978801,0.044968,0.890666,1.066936,0.000000e+00


### Extras

In [14]:
# demeaned in action (x1 unconditional, x2 conditional on id1)
fr.ols(y=R.y, x=I+D.x1+D.x2('id1'), data=data)

y,coeff,stderr,low95,high95,pvalue
I,1.048618,0.003414,1.041927,1.055310,0.0
x1-μ,0.295601,0.003419,0.288900,0.302302,0.0
x2-μ|id1,0.592314,0.003398,0.585653,0.598974,0.0


In [15]:
# quantile binning (x2 on deciles, self interacted)
fr.ols(y=R.y, x=I+R.x1+B.x2(10)*R.x2, data=data)

y,coeff,stderr,low95,high95,pvalue
I,1.032759,0.006341,1.020331,1.045187,0.0
x1,0.295520,0.003422,0.288814,0.302226,0.0
(x2:bin10=1)*x2,0.562329,0.020962,0.521244,0.603414,0.0
(x2:bin10=2)*x2,0.588047,0.010317,0.567827,0.608267,0.0
(x2:bin10=3)*x2,0.578670,0.008322,0.562358,0.594982,0.0
(x2:bin10=4)*x2,0.552255,0.012898,0.526977,0.577534,0.0
(x2:bin10=5)*x2,0.612026,0.025225,0.562585,0.661467,0.0
(x2:bin10=6)*x2,0.611378,0.010249,0.591289,0.631466,0.0
(x2:bin10=7)*x2,0.605579,0.008398,0.589118,0.622040,0.0
(x2:bin10=8)*x2,0.609507,0.012284,0.585431,0.633583,0.0


In [16]:
# hdfe — fast for one categorical term
fr.ols(y=R.y, x=I+R.x1+R.x2, hdfe=C.id1*C.id2, data=data)

y,coeff,stderr,low95,high95,pvalue
I,0.158384,0.106767,-0.050876,0.367643,0.137954
x1,0.295242,0.003188,0.288993,0.301491,0.000000
x2,0.593225,0.003168,0.587015,0.599434,0.000000
"id1=A,id2=1",-0.068296,0.143535,-0.349619,0.213026,0.634205
"id1=A,id2=2",0.059658,0.154722,-0.243593,0.362908,0.699808
...,...,...,...,...,...
"id1=J,id2=95",1.515430,0.142673,1.235796,1.795064,0.000000
"id1=J,id2=96",1.640398,0.152318,1.341859,1.938936,0.000000
"id1=J,id2=97",1.525558,0.145391,1.240597,1.810518,0.000000
"id1=J,id2=98",1.778260,0.144439,1.495165,2.061355,0.000000


In [17]:
# absorbtion — fast but no std errors
fr.ols(y=R.y, x=I+R.x1+R.x2, absorb=C.id1*C.id2, data=data)

y,coeff,stderr,low95,high95,pvalue
I,1.049263,0.002852,1.043674,1.054853,0.0
x1,0.295420,0.003129,0.289287,0.301553,0.0
x2,0.593774,0.003251,0.587402,0.600146,0.0


In [18]:
# clustered standard errors (any term allowed)
fr.ols(y=R.y, x=I+R.x1+R.x2+C.id1+C.id2, cluster=C.id1*C.id2, data=data)

y,coeff,stderr,low95,high95,pvalue
I,0.152789,0.027124,0.099627,0.205951,1.771259e-08
x1,0.295420,0.003129,0.289287,0.301553,0.000000e+00
x2,0.593774,0.003251,0.587402,0.600147,0.000000e+00
id1=B,0.071617,0.011754,0.048580,0.094655,1.108260e-09
id1=C,0.167662,0.012462,0.143238,0.192086,0.000000e+00
...,...,...,...,...,...
id2=95,0.863448,0.038707,0.787583,0.939313,0.000000e+00
id2=96,0.950444,0.036558,0.878791,1.022096,0.000000e+00
id2=97,0.961423,0.043568,0.876032,1.046814,0.000000e+00
id2=98,0.978801,0.036580,0.907106,1.050496,0.000000e+00


### GLM

In [ ]:
fr.poisson(y=R.p, x=I+R.x1+R.x2+C.id1+C.id2, data=data)

In [ ]:
fr.poisson(y=R.p, x=I+R.x1+R.x2+C.id1, hdfe=C.id2, data=data)

### Accuracy

In [ ]:
# how do we do on ids?
fr.testing.test_ols(data, plot=True);

In [ ]:
# slow on first run, try it twice
fr.testing.test_glm(data, plot=True);

### Custom Factors

In [ ]:
# simple factor
@factor
def demean(x):
    return x - np.mean(x)
fact = demean.x1
print(fact)
fact.eval(data)

In [ ]:
# complex factor
@factor(eval_args=(0, 1))
def cdemean(x, i):
    datf = pd.DataFrame({'vals': x, 'cond': i})
    cmean = datf.groupby('cond')['vals'].mean().rename('mean')
    datf = datf.join(cmean, on='cond')
    return datf['vals'] - datf['mean']
cfact = cdemean('x1', 'id1')
print(cfact)
cfact.eval(data)

In [ ]:
# named factor
@factor(name=lambda x: f'{x}-μ')
def ndemean(x):
    return x - np.mean(x)
nfact = ndemean.x1
print(nfact)
nfact.eval(data)

In [ ]:
# inject into eval namespace (note @ required to reference)
logit = lambda x: 1/(1+np.exp(-x))
lfact = R('@logit(x1)')
print(lfact)
lfact.eval(data, extern={'logit': logit})

In [ ]:
# create custom factor (somewhat more elegant)
flogit = factor(logit, name='logit')
lfact = flogit('x1')
print(lfact)
lfact.eval(data)